<font color='darkred'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Exercises

For these exercises, you'll be creating a [Python class](https://www.hackerearth.com/practice/python/object-oriented-programming/classes-and-objects-i/tutorial/) in the *apputil.py* file.

A few tips:

- Some of these exercises will require some "JSON digging".
- <font color='lightblue'>Your functions should still adhere to [best practices](https://codesignal.com/learn/courses/clean-code-basics-with-python/lessons/clean-function-design-in-python), so **your class should probably contain more functions than only the ones dictated below**.</font>


## Exercise 1

Create a Python class named `Genius` such that the following code initializes the object, and "saves" the access token as an attribute of the object. You'll need to use this attribute for Exercises 2 and 3.

```python
from apputil import Genius

genius = Genius(access_token="access_token")
```

In [ ]:

# Define the Genius class
class Genius:
    """Class"""

    def __init__(self, access_token):
        """
        Initialize the Genius class.
        """
        # Save the access token
        self.access_token = access_token

        # Print for confirmation
        print(f"Genius initialized with access token: {self.access_token}")


# Test the Genius class
if __name__ == "__main__":
    # Create a Genius object
    genius = Genius(access_token="access_token")

    # Verify the access token was saved
    print(genius.access_token)


Genius initialized with access token: access_token
access_token


## Exercise 2

Reference the `json_data` in the lab. Notice, when we search for an artist name (e.g., "Missy Elliot") in Genius, the result is a list of *songs* attributed to that artist. Suppose we want to capture information about the artist themselves.

Create a method for our `Genius` class called `.get_artist(search_term)` which does the following:

1. Extract the (most likely, "Primary") Artist ID from the first "hit" of the `search_term`.
2. Use the [API path](https://docs.genius.com/#artists-h2) for this Artist ID to pull information about the artist.
3. **Return** the dictionary containing the resulting JSON object.

For example, the following code should return a dictionary of artist information:

```python
genius.get_artist("Radiohead")
```

In [5]:
import requests

class Genius:
    """Class"""

    def __init__(self, access_token):
        """
        Initialize the Genius class.
        """
        self.access_token = access_token
        self.base_url = "http://api.genius.com"

    def get_artist(self, search_term):
        """
        Search for an artist and return their information as a dictionary.
        """
    
        genius_search_url = f"{self.base_url}/search?q={search_term}&access_token={self.access_token}"
        response = requests.get(genius_search_url)

        if response.status_code != 200:
            raise Exception(f"Search request failed: {response.status_code} - {response.text}")
        search_results = response.json()
        try:
            first_hit = search_results["response"]["hits"][0]["result"]
            artist_id = first_hit["primary_artist"]["id"]
        except (IndexError, KeyError):
            raise Exception("No artist found for that search term.")
        artist_url = f"{self.base_url}/artists/{artist_id}?access_token={self.access_token}"
        artist_response = requests.get(artist_url)

        if artist_response.status_code != 200:
            raise Exception(f"Artist request failed: {artist_response.status_code} - {artist_response.text}")
        return artist_response.json()


# Example usage
if __name__ == "__main__":

    genius = Genius(access_token=ACCESS_TOKEN)
    artist_info = genius.get_artist("Radiohead")

    # Print test
    print(artist_info["response"]["artist"]["name"])

Exception: Search request failed: 401 - {"error":"invalid_token","error_description":"The access token provided is expired, revoked, malformed or invalid for other reasons."}

## Exercise 3

Use the result from Exercise 2 to create another method for our `Genius` class called `.get_artists(search_terms)` (plural) which takes in a *list* of search terms, and returns a DataFrame containing a row for each search term, and the following columns:

- `search_term`: the raw search term from `search_terms`
- `artist_name`: the (most likely) artist name for the search term
- `artist_id`: the Genius Artist ID for that artist, based on the API call
- `followers_count`: the number of followers for that artist (if available)

For example, the following should return a DataFrame with 4 rows:

```python
genius.get_artists(['Rihanna', 'Tycho', 'Seal', 'U2'])
```

In [8]:
import requests
import pandas as pd

class Genius:
    """Class"""

    def __init__(self, access_token):
        """
        Initialize the Genius class
        """
        self.access_token = access_token
        self.base_url = "http://api.genius.com"

    def get_artist(self, search_term):
        """
        Search for an artist
        """
        genius_search_url = f"{self.base_url}/search?q={search_term}&access_token={self.access_token}"
        response = requests.get(genius_search_url)

        if response.status_code != 200:
            raise Exception(f"Search request failed: {response.status_code} - {response.text}")

        search_results = response.json()
        try:
            first_hit = search_results["response"]["hits"][0]["result"]
            artist_id = first_hit["primary_artist"]["id"]
        except (IndexError, KeyError):
            raise Exception(f"No artist found for search term '{search_term}'.")
        artist_url = f"{self.base_url}/artists/{artist_id}?access_token={self.access_token}"
        artist_response = requests.get(artist_url)

        if artist_response.status_code != 200:
            raise Exception(f"Artist request failed: {artist_response.status_code} - {artist_response.text}")

        return artist_response.json()

    def get_artists(self, search_terms):
        """
        Take in a list of search terms 
        """
        data = []

        for term in search_terms:
            try:
                artist_data = self.get_artist(term)
                artist_info = artist_data["response"]["artist"]

                data.append({
                    "search_term": term,
                    "artist_name": artist_info.get("name"),
                    "artist_id": artist_info.get("id"),
                    "followers_count": artist_info.get("followers_count", None)
                })

            except Exception as e:
                data.append({
                    "search_term": term,
                    "artist_name": None,
                    "artist_id": None,
                    "followers_count": None
                })
                print(f"Error retrieving artist '{term}': {e}")

        # Create DataFrame
        df = pd.DataFrame(data, columns=["search_term", "artist_name", "artist_id", "followers_count"])
        return df


# Example usage
if __name__ == "__main__":
   
    genius = Genius(access_token=ACCESS_TOKEN)
    df = genius.get_artists(['Rihanna', 'Tycho', 'Seal', 'U2'])

    print(df)

Error retrieving artist 'Rihanna': Search request failed: 401 - {"error":"invalid_token","error_description":"The access token provided is expired, revoked, malformed or invalid for other reasons."}
Error retrieving artist 'Tycho': Search request failed: 401 - {"error":"invalid_token","error_description":"The access token provided is expired, revoked, malformed or invalid for other reasons."}
Error retrieving artist 'Seal': Search request failed: 401 - {"error":"invalid_token","error_description":"The access token provided is expired, revoked, malformed or invalid for other reasons."}
Error retrieving artist 'U2': Search request failed: 401 - {"error":"invalid_token","error_description":"The access token provided is expired, revoked, malformed or invalid for other reasons."}
  search_term artist_name artist_id followers_count
0     Rihanna        None      None            None
1       Tycho        None      None            None
2        Seal        None      None            None
3     

## Bonus Exercise (optional)

1. Gather a list of 100+ various musical artists, and save this list in a TXT file.
2. Write a Python *script* that saves the result of `.get_artists` for these artists in a CSV file.
3. If you have time, adjust this script to use multiprocessing.